## 준비하기
1. 아래 예제를 실행하기 위해서는 사전에 ./setup.sh 파일을 실행하거나 ./start_notebook.sh 로 노트북을 구동하여 의존성 라이브러리들을 미리 설치해두셔야 합니다.
2. qdrant서버가 떠 있어야 정상 동작합니다. ./start_qdrant.sh 를 실행하여 qdrant 서버를 띄워주세요.
3. openapi 키가 있어야 합니다. 

## 목적:
- Qdrant를 활용하여 Retrieval-Augmented Generation(RAG) 실습
- 벡터 검색을 통해 GPT 모델의 응답을 향상시킴

## 왜 필요한가?
- 단순한 질의응답 시스템보다 정확한 정보를 제공할 수 있음
- AI 챗봇이나 검색 시스템을 개선하는 데 활용 가능

## 주요 개념:
- **RAG (Retrieval-Augmented Generation)**: 검색과 생성 모델을 결합하여 더 정확한 응답을 제공하는 기술
- **Qdrant**: 대규모 벡터 데이터를 검색하는 데 최적화된 벡터 데이터베이스
- **LLM (Large Language Model)**: 대규모 자연어 처리 모델, 예: GPT

In [1]:
# 라이브러리 로딩 (아래 부분에서 오류가 발생하면 pip -r ./requirements.txt 로 의존성을 설치해주세요)
import os
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from langchain_huggingface import HuggingFaceEmbeddings

In [4]:
# Qdrant 연결
qdrant_client = QdrantClient(host="localhost", port=6333)
collection_name = "ex5_sample_vectors"

# Qdrant 컬렉션 생성 (없을 경우 자동 생성)
if not qdrant_client.collection_exists(collection_name):
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=384, distance=Distance.COSINE)  # 384는 임베딩 차원 수
    )

In [5]:
# 임베딩 모델 로드
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [6]:
# Qdrant 기반 벡터 저장소 생성
vectorstore = QdrantVectorStore(
    client=qdrant_client,
    collection_name=collection_name,
    embedding=embeddings
)

In [7]:
# OpenAI API 키 설정 (환경 변수에서 가져오거나 기본값 설정)
openai_api_key = os.environ.get("OPENAI_API_KEY", "your_api_key")

# LLM 모델 (GPT API) 설정
llm = OpenAI(api_key=openai_api_key)

In [8]:
# RAG 체인 생성
retriever = vectorstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

In [10]:
# 질문 실행
question = "오늘의 날씨를 알려줘"
response = qa_chain.invoke({"query": question})

# 응답 출력
print("LLM 응답:", response)

LLM 응답: {'query': '오늘의 날씨를 알려줘', 'result': "\nI'm sorry, I do not have information on the weather for today."}


In [ ]:
## todo: 데이터 넣고 결과가 변경된것을 보여야함